# READ ME!!!

1) Please read the report first.

2) Check [this](https://docs.google.com/document/d/1MBKA2ERi1SLBhWiGCeB-u2QcmEVVCv83jBJg9BamhKw/edit?usp=sharing) file to continue.



# Data Upload & Imports

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

import torch as torch

import tensorflow_datasets as tfds
import random 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.util import montage

import pandas as pd
import cv2
import keras
import numpy as np


from sklearn.metrics import accuracy_score
from tqdm.autonotebook import tqdm

#kaya's
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
import os
from glob import glob

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model, Input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet201
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import ResNet152V2
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.xception import Xception

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping


%matplotlib inline
import matplotlib.pyplot as plt

In [0]:
def see_data(i):
    print("---Showing img #" + str(i) +"---")
    print("Shape: " + str(training_data[i][0].shape))
    print("Label: " +  str(training_data[i][1] + 1))
    print("Array: " + str(training_data[i][0]))
    plt.imshow(training_data[i][0])
    plt.show()
    print("---Show done---")

def see_data_t(i):
    print("---Showing img #" + str(i) +"---")
    print("Shape: " + str(training_data[i][0].shape))
    print("Array: " + str(training_data[i][0]))
    plt.imshow(training_data[i][0])
    plt.show()
    print("---Show done---")

def print_perc(l):
    c1=0
    c2=0
    c3=0
    c4=0
    c5=0
    total = len(l)
    for label in l:
        if(label == 0):
            c1 += 1
        if(label == 1):
            c2 += 1
        if(label == 2):
            c3 += 1
        if(label == 3):
            c4 += 1
        if(label == 4):
            c5 += 1
    
    print("Percentage of 1: %" + str(format( c1/total*100, '.2f') ) )
    print("Percentage of 2: %" + str(format( c2/total*100, '.2f') ) )
    print("Percentage of 3: %" + str(format( c3/total*100, '.2f') ) )
    print("Percentage of 4: %" + str(format( c4/total*100, '.2f') ) )
    print("Percentage of 5: %" + str(format( c5/total*100, '.2f') ) )

def ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    y = layers.Average()(outputs)
    model = Model(model_input, y, name='ensemble')
    return model


# Defining Pre-Trained Models & Creating Ensemble Model

In [0]:
import pickle
from itertools import combinations
import copy

IMG_SIZE = 128

#Please change the path for your own drive path and assign accordingly -> use pickled database for this

Training_Data_Trimmed = "/content/drive/My Drive/CS412 PROJECT/Final Notebooks/pickle/Training_data_trimmed.pickle"
Training_Data_Trimmed_Cropped = "/content/drive/My Drive/CS412 PROJECT/Final Notebooks/pickle/Training_data_trimmed_crop.pickle"

Testing_Data_Trimmed = "/content/drive/My Drive/CS412 PROJECT/Final Notebooks/pickle/Testing_data_trimmed.pickle"
Testing_Data_Trimmed_Cropped = "/content/drive/My Drive/CS412 PROJECT/Final Notebooks/pickle/Testing_data_trimmed_crop.pickle"

#DONT CHANGE THIS PART
weight_trimmed = "_weights.h5"
weight_trimmed_cropped = "_weights_crop.h5"


In [0]:
#Please choose a data-type to continue && Set the myWeight variable accordingly
#For the Final Submission we used Trimmed_Cropped Data!!!

database = Training_Data_Trimmed_Cropped

test_database = Testing_Data_Trimmed_Cropped

myWeight = weight_trimmed_cropped

In [0]:
#There are many different path options here. Please be careful!!!

if(database == Training_Data_Trimmed and myWeight == weight_trimmed and test_database == Testing_Data_Trimmed):
    print("---Using Trimmed Data Only---")
elif(database == Training_Data_Trimmed_Cropped and myWeight == weight_trimmed_cropped and test_database == Testing_Data_Trimmed_Cropped):
    print("---Using Trimmed And Cropped Data---")
else:
    print("ERROR WITH THE DATA TYPES!!! DO NOT CONTINUE. PLEASE CHECK DATA TYPES!!!")

---Using Trimmed And Cropped Data---


In [0]:
#STARTING THE ENSEMBLE MODEL PLEASE BE PATIENT TAKES A WHILE :)

print("Section 1: Prepare the Database: ", database, myWeight)
training_data = []
training_data = pickle.load( open(database,"rb") )

print("\t--> Database Loaded!")

X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)   #gray scale 1 --> 3 for RGB
y = np.array(y)

X = X/255.0

X, valid_X, y, valid_y = train_test_split(X, y, test_size = 0.1, random_state=3) # used train test split method

print("\t--> Data Shapes: ")
print("\t\t", X.shape,y.shape,valid_X.shape,valid_y.shape)

valid_y = to_categorical(valid_y,5)
print("\t--> To_Categorical done!")
print("\t\t", valid_y.shape)

print("Section 2: Defining the Models")

input_shape = valid_X[0,:,:,:].shape
model_input = Input(shape=input_shape)
    
training_data = []
X = []
y = []

print("\t--> Defining InceptionV3")
inceptionx = InceptionV3(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)
    
for layer in inceptionx.layers:
    layer.trainable = True

inception_last_layer = inceptionx.get_layer('mixed10')
#print('last layer output shape:', inception_last_layer.output_shape)
inception_last_output = inception_last_layer.output

# Flatten the output layer to 1 dimension
x_inception = layers.GlobalMaxPooling2D()(inception_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_inception = layers.Dense(512, activation='relu')(x_inception)
# Add a dropout rate of 0.7
x_inception = layers.Dropout(0.5)(x_inception)
# Add a final sigmoid layer for classification
x_inception = layers.Dense(5, activation='softmax')(x_inception)

# Configure and compile the model

#inception_model --> Hawkeye
Hawkeye_model = Model(model_input, x_inception)
optimizer = Adam(lr=0.0001, decay=0.0, amsgrad=True)
Hawkeye_model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

path = "/content/drive/My Drive/CS412 PROJECT/Final Notebooks/weights/InceptionV3_model"
Hawkeye_model.load_weights( path + myWeight )

print("\t--> Hawkeye Defined!")
#################################
print("\t--> Defining DenseNet201")

denseNetx = DenseNet201(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

for layer in denseNetx.layers:
    layer.trainable = True

denseNet_last_layer = denseNetx.get_layer('relu')
#print('last layer output shape:', denseNet_last_layer.output_shape)
denseNet_last_output = denseNet_last_layer.output

# Flatten the output layer to 1 dimension
x_denseNet = layers.GlobalMaxPooling2D()(denseNet_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_denseNet = layers.Dense(512, activation='relu')(x_denseNet)
# Add a dropout rate of 0.7
x_denseNet = layers.Dropout(0.5)(x_denseNet)
# Add a final sigmoid layer for classification
x_denseNet = layers.Dense(5, activation='softmax')(x_denseNet)

#Configure and compile the model

Thor_model = Model(model_input, x_denseNet)
#optimizer = Adam(lr=0.0001, decay=0.0, amsgrad=True)
Thor_model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

path = "/content/drive/My Drive/CS412 PROJECT/Final Notebooks/weights/DenseNet_model"
Thor_model.load_weights( path + myWeight )

print("\t--> Thor Defined!")
#################################
print("\t--> Defining InceptionResNet")

inceptionResNetV2x = InceptionResNetV2(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

for layer in inceptionResNetV2x.layers:
    layer.trainable = True

inceptionResNetV2_last_layer = inceptionResNetV2x.get_layer('conv_7b_ac')
#print('last layer output shape:', inceptionResNetV2_last_layer.output_shape)
inceptionResNetV2_last_layer_output = inceptionResNetV2_last_layer.output

# Flatten the output layer to 1 dimension
x_inceptionResNetV2 = layers.GlobalMaxPooling2D()(inceptionResNetV2_last_layer_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_inceptionResNetV2 = layers.Dense(512, activation='relu')(x_inceptionResNetV2)
# Add a dropout rate of 0.7
x_inceptionResNetV2 = layers.Dropout(0.5)(x_inceptionResNetV2)
# Add a final sigmoid layer for classification
x_inceptionResNetV2 = layers.Dense(5, activation='softmax')(x_inceptionResNetV2)

# Configure and compile the model

CaptainAmerica_model = Model(model_input, x_inceptionResNetV2)
#optimizer = Adam(lr=0.0001, decay=0.0, amsgrad=False)
CaptainAmerica_model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

path = "/content/drive/My Drive/CS412 PROJECT/Final Notebooks/weights/InceptionResNetV2_model"
CaptainAmerica_model.load_weights( path + myWeight )

print("\t--> CaptainAmerica Defined")
#################################
print("\t--> Defining InceptionV3-FineTune")

inception2x = InceptionV3(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

for layer in inception2x.layers:
    layer.trainable = True

inception2_last_layer = inception2x.get_layer('mixed10')
#print('last layer output shape:', inception2_last_layer.output_shape)
inception2_last_output = inception2_last_layer.output

# Flatten the output layer to 1 dimension
x_inception2 = layers.GlobalMaxPooling2D()(inception2_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_inception2 = layers.Dense(512, activation='relu')(x_inception2)
# Add a dropout rate of 0.7
x_inception2 = layers.Dropout(0.5)(x_inception2)
# Add a final sigmoid layer for classification
x_inception2 = layers.Dense(5, activation='softmax')(x_inception2)

# Configure and compile the model

Wanda_model = Model(model_input, x_inception2)
#optimizer = Adam(lr=0.0001, decay=0.0, amsgrad=False)
Wanda_model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

path = "/content/drive/My Drive/CS412 PROJECT/Final Notebooks/weights/InceptionV3_fine_model"
Wanda_model.load_weights( path + myWeight )

print("\t--> Wanda Defined")
#################################
print("\t--> Defining DenseNet201-FineTune")

denseNet2x = DenseNet201(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

for layer in denseNet2x.layers:
    layer.trainable = True

denseNet2_last_layer = denseNet2x.get_layer('relu')
#print('last layer output shape:', denseNet2_last_layer.output_shape)
denseNet2_last_output = denseNet2_last_layer.output

# Flatten the output layer to 1 dimension
x_denseNet2 = layers.GlobalMaxPooling2D()(denseNet2_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_denseNet2 = layers.Dense(512, activation='relu')(x_denseNet2)
# Add a dropout rate of 0.7
x_denseNet2 = layers.Dropout(0.5)(x_denseNet2)
# Add a final sigmoid layer for classification
x_denseNet2 = layers.Dense(5, activation='softmax')(x_denseNet2)

# Configure and compile the model

Hulk_model = Model(model_input, x_denseNet2)
#optimizer = Adam(lr=0.0001, decay=0.0, amsgrad=False)
Hulk_model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

path = "/content/drive/My Drive/CS412 PROJECT/Final Notebooks/weights/DenseNet_fine_model"
Hulk_model.load_weights( path + myWeight )

print("\t--> Hulk Defined")
#################################
print("\t--> Defining Xception")

Xceptionx = Xception(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

for layer in Xceptionx.layers:
    layer.trainable = True

Xception_last_layer = Xceptionx.get_layer('block14_sepconv2_act')
#print('last layer output shape:', last_layer.output_shape)
Xception_last_output = Xception_last_layer.output

# Flatten the output layer to 1 dimension
x_Xception = layers.GlobalMaxPooling2D()(Xception_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_Xception = layers.Dense(512, activation='relu')(x_Xception)
# Add a dropout rate of 0.7
x_Xception = layers.Dropout(0.5)(x_Xception)
# Add a final sigmoid layer for classification
x_Xception = layers.Dense(5, activation='softmax')(x_Xception)

# Configure and compile the model

IronMan_model = Model(model_input, x_Xception)
#optimizer = Adam(lr=0.0001, decay=0.0, amsgrad=False)
IronMan_model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

path = "/content/drive/My Drive/CS412 PROJECT/Final Notebooks/weights/Xception_model"
IronMan_model.load_weights( path + myWeight )

print("\t--> IronMan Defined")
#################################
print("\t--> Defining XceptionFine-Tune")
    
Xception2x = Xception(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)

for layer in Xception2x.layers:
    layer.trainable = True

Xception2_last_layer = Xception2x.get_layer('block14_sepconv2_act')
#print('last layer output shape:', last_layer.output_shape)
Xception2_last_output = Xception2_last_layer.output

# Flatten the output layer to 1 dimension
x_Xception2 = layers.GlobalMaxPooling2D()(Xception2_last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x_Xception2 = layers.Dense(512, activation='relu')(x_Xception2)
# Add a dropout rate of 0.7
x_Xception2 = layers.Dropout(0.5)(x_Xception2)
# Add a final sigmoid layer for classification
x_Xception2 = layers.Dense(5, activation='softmax')(x_Xception2)

# Configure and compile the model

Groot_model = Model(model_input, x_Xception2)
#optimizer = Adam(lr=0.0001, decay=0.0, amsgrad=False)
Groot_model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

path = "/content/drive/My Drive/CS412 PROJECT/Final Notebooks/weights/Xception_fine_model"
Groot_model.load_weights( path + myWeight )

print("\t--> Groot Defined")

print("Section 3: Assembling Avengers!!!")
    
our_models = {
    "DenseNet": Thor_model,
    "DenseNet-Fine": Hulk_model,
    "Inception": Hawkeye_model,
    "Inception-Fine": Wanda_model,
    "InceptionResNet": CaptainAmerica_model,
    "Xception" : IronMan_model,
    "Xception-Fine": Groot_model,
}

Section 1: Prepare the Database:  /content/drive/My Drive/CS412 PROJECT/Final Notebooks/pickle/Training_data_trimmed_crop.pickle _weights_crop.h5
	--> Database Loaded!
	--> Data Shapes: 
		 (9000, 128, 128, 3) (9000,) (1000, 128, 128, 3) (1000,)
	--> To_Categorical done!
		 (1000, 5)
Section 2: Defining the Models
	--> Defining InceptionV3
	--> Hawkeye Defined!
	--> Defining DenseNet201
	--> Thor Defined!
	--> Defining InceptionResNet
	--> CaptainAmerica Defined
	--> Defining InceptionV3-FineTune
	--> Wanda Defined
	--> Defining DenseNet201-FineTune
	--> Hulk Defined
	--> Defining Xception
	--> IronMan Defined
	--> Defining XceptionFine-Tune
	--> Groot Defined
Section 3: Assembling Avengers!!!


In [0]:
#Available Models to Use

ModelsAll = [
    our_models["DenseNet"],
    our_models["DenseNet-Fine"],
    our_models["Inception"],
    our_models["Inception-Fine"],
    our_models["InceptionResNet"],
    our_models["Xception"],
    our_models["Xception-Fine"],
]

print(len(ModelsAll), 2**len(ModelsAll))

7 128


This is how you can ensemble a model. The function is defined above as ensemble

In [0]:
#This Model Combination is the Final Version

ensemble_model = ensemble( [ our_models["DenseNet"], our_models["Xception"], our_models["InceptionResNet"], our_models["Inception-Fine"]], model_input)

ensemble_model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

loss_val, acc_val = ensemble_model.evaluate(valid_X, valid_y, verbose=1)
print("\t--> Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))

1000/1000 [==============================] - 37s 37ms/step
	--> Validation: accuracy = 0.821000  ;  loss_v = 0.530192



# Please Do Not Run this section, this section is for logbook

We used this section as a log book to keep track of the models we trained so far.

In [0]:
# Trimmed Models Accuracies
# DenseNet201: 
# Train Acc: 95.05 / Valid Acc: 78.7

# DenseNet201 Fine:
# Train Acc: 93.48 / Valid Acc: 77.4

# InceptionV3:
# Train Acc: 71.82 / Valid Acc: 70.39

# InceptionV3 Fine:
# Train Acc: 82.55 / Valid Acc: 71.2

# Xception:
# Train Acc: 95.53 / Valid Acc: 78.0

# Xception Fine:
# Train Acc: 92.27 / Valid Acc: 74.8

# Inception resnet_1:
# Train Acc: 88.13 / Valid Acc: 75.42

# Inception resnet_2:
# Train Acc: 89.91 / Valid Acc: 74.0

# Trimmed Cropped Models Accuracies
# Xception:
# Train Acc: 96.44 / 78.5

# DenseNet201:
# Train Acc: 94.4 / Valid Acc: 77.4

# InceptionV3:
# Train Acc: 64.35 / Valid Acc: 65.1

# Xception Fine:
# Train Acc: 95.06 / Valid Acc: 79.0

# DenseNet201 Fine:
# Train Acc: 93.86 / Valid Acc: 75.9

# InceptionV3 Fine:
# Train Acc: 84.64 / Valid Acc: 72.2

# InceptionResNet:
#Train Acc: 85.78 / Valid Acc: 71.49


In [0]:
# # TRIMMED CROPPED!!!
# tryModels = [
            
#             #2's
#             [ our_models["DenseNet"], our_models["Xception"] ],                                                                         #0 Valid: 81.5

#             [ our_models["DenseNet"], our_models["Xception-Fine"] ],                                                                    #1 Valid: 81.2

#             [ our_models["DenseNet"], our_models["InceptionResNet"] ],                                                                  #2 Valid: 79.4

#             [ our_models["DenseNet"], our_models["Inception"] ],                                                                        #3 Valid: 79.3

#             [ our_models["DenseNet"], our_models["Inception-Fine"] ],                                                                   #4 Valid: 79.5

#             #3's
#             [ our_models["DenseNet"], our_models["Xception"], our_models["InceptionResNet"]],                                           #5 Valid: 81.8 -->Submitted
             
#             [ our_models["DenseNet"], our_models["Xception-Fine"], our_models["InceptionResNet"]],                                      #6 Valid: 81.0
            
#             [ our_models["DenseNet"], our_models["Xception"], our_models["Inception"]],                                                 #7 Valid: 81.9
            
#             [ our_models["DenseNet"], our_models["Xception"], our_models["Inception-Fine"]],                                            #8 Valid: 82.3 --> Submitted

#             [ our_models["DenseNet"], our_models["Xception-Fine"], our_models["Inception"]],                                            #9 Valid: 82.3
            
#             [ our_models["DenseNet"], our_models["Xception-Fine"], our_models["Inception-Fine"]],                                       #10 Valid: 82.2
            
#             [ our_models["DenseNet"], our_models["Inception"], our_models["InceptionResNet"]],                                          #11 Valid: 79.4
             
#             [ our_models["DenseNet"], our_models["Inception-Fine"], our_models["InceptionResNet"]],                                     #12 Valid: 80.2
            
#             #4's
#             [ our_models["DenseNet"], our_models["Xception"], our_models["InceptionResNet"], our_models["Inception"]],                  #13 Valid: 82.2
             
#             [ our_models["DenseNet"], our_models["Xception"], our_models["InceptionResNet"], our_models["Inception-Fine"]],             #14 Valid: 82.1
             
#             [ our_models["DenseNet"], our_models["Xception-Fine"], our_models["InceptionResNet"], our_models["Inception"]],             #15 Valid: 81.3

#             [ our_models["DenseNet"], our_models["Xception-Fine"], our_models["InceptionResNet"], our_models["Inception-Fine"]],        #16 Valid: 81.5
            
#             #DenseNetFine 2's
#             [ our_models["DenseNet-Fine"], our_models["Xception"] ],                                                                    #17 Valid: 80.5

#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"] ],                                                               #18 Valid: 80.6

#             [ our_models["DenseNet-Fine"], our_models["InceptionResNet"] ],                                                             #19 Valid: 77.8

#             [ our_models["DenseNet-Fine"], our_models["Inception"] ],                                                                   #20 Valid: 78.2

#             [ our_models["DenseNet-Fine"], our_models["Inception-Fine"] ],                                                              #21 Valid: 78.7

#             #DenseNetFine 3's
#             [ our_models["DenseNet-Fine"], our_models["Xception"], our_models["InceptionResNet"]],                                      #22 Valid: 80.9

#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"], our_models["InceptionResNet"]],                                 #23 Valid: 80.4

#             [ our_models["DenseNet-Fine"], our_models["Xception"], our_models["Inception"]],                                            #24 Valid: 80.9
            
#             [ our_models["DenseNet-Fine"], our_models["Xception"], our_models["Inception-Fine"]],                                       #25 Valid: 81.3

#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"], our_models["Inception"]],                                       #26 Valid: 80.9
            
#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"], our_models["Inception-Fine"]],                                  #27 Valid: 81.4
            
#             [ our_models["DenseNet-Fine"], our_models["Inception"], our_models["InceptionResNet"]],                                     #28 Valid: 78.2
             
#             [ our_models["DenseNet-Fine"], our_models["Inception-Fine"], our_models["InceptionResNet"]],                                #29 Valid: 78.9

#             #DenseNetFine 4's
#             [ our_models["DenseNet-Fine"], our_models["Xception"], our_models["InceptionResNet"], our_models["Inception"]],             #30 Valid: 81.4
             
#             [ our_models["DenseNet-Fine"], our_models["Xception"], our_models["InceptionResNet"], our_models["Inception-Fine"]],        #31 Valid: 80.9
             
#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"], our_models["InceptionResNet"], our_models["Inception"]],        #32 Valid: 80.5

#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"], our_models["InceptionResNet"], our_models["Inception-Fine"]],   #33 Valid: 80.7

#         ]

# Valid_Acc = []

# for i in range(len(tryModels)):
#     ChoosenModel = i

#     ensemble_model = ensemble( tryModels[ChoosenModel], model_input)
#     ensemble_model.compile(loss='categorical_crossentropy',
#                 optimizer=optimizer,
#                 metrics=['accuracy'])

#     loss_val, acc_val = ensemble_model.evaluate(valid_X, valid_y, verbose=1)
#     print("Model #%d --> Validation: accuracy = %f  ;  loss_v = %f" % (ChoosenModel, acc_val, loss_val))
    
#     Valid_Acc.append(acc_val)

In [0]:
# for i in range(len(Valid_Acc)):
#     print("Model:" + str(i) + " --> " + str(Valid_Acc[i])[0:6])

Model:0 --> 0.8149
Model:1 --> 0.8119
Model:2 --> 0.7940
Model:3 --> 0.7929
Model:4 --> 0.7950
Model:5 --> 0.8180
Model:6 --> 0.8100
Model:7 --> 0.8190
Model:8 --> 0.8230
Model:9 --> 0.8230
Model:10 --> 0.8220
Model:11 --> 0.7940
Model:12 --> 0.8019
Model:13 --> 0.8220
Model:14 --> 0.8209
Model:15 --> 0.8130
Model:16 --> 0.8149
Model:17 --> 0.8050
Model:18 --> 0.8059
Model:19 --> 0.7779
Model:20 --> 0.7820
Model:21 --> 0.7870
Model:22 --> 0.8090
Model:23 --> 0.8040
Model:24 --> 0.8090
Model:25 --> 0.8130
Model:26 --> 0.8090
Model:27 --> 0.8140
Model:28 --> 0.7820
Model:29 --> 0.7889
Model:30 --> 0.8140
Model:31 --> 0.8090
Model:32 --> 0.8050
Model:33 --> 0.8069


In [0]:
# Valid_Acc_Trimmed_Cropped = copy.deepcopy(Valid_Acc)

In [0]:
# # TRIMMED ONLY!!!
# tryModels = [
            
#             #2's
#             [ our_models["DenseNet"], our_models["Xception"] ],                                                                         #0 Valid: 81.5

#             [ our_models["DenseNet"], our_models["Xception-Fine"] ],                                                                    #1 Valid: 81.2

#             [ our_models["DenseNet"], our_models["InceptionResNet"] ],                                                                  #2 Valid: 79.4

#             [ our_models["DenseNet"], our_models["Inception"] ],                                                                        #3 Valid: 79.3

#             [ our_models["DenseNet"], our_models["Inception-Fine"] ],                                                                   #4 Valid: 79.5

#             #3's
#             [ our_models["DenseNet"], our_models["Xception"], our_models["InceptionResNet"]],                                           #5 Valid: 81.8 -->Submitted
             
#             [ our_models["DenseNet"], our_models["Xception-Fine"], our_models["InceptionResNet"]],                                      #6 Valid: 81.0
            
#             [ our_models["DenseNet"], our_models["Xception"], our_models["Inception"]],                                                 #7 Valid: 81.9
            
#             [ our_models["DenseNet"], our_models["Xception"], our_models["Inception-Fine"]],                                            #8 Valid: 82.3

#             [ our_models["DenseNet"], our_models["Xception-Fine"], our_models["Inception"]],                                            #9 Valid: 82.3
            
#             [ our_models["DenseNet"], our_models["Xception-Fine"], our_models["Inception-Fine"]],                                       #10 Valid: 82.2
            
#             [ our_models["DenseNet"], our_models["Inception"], our_models["InceptionResNet"]],                                          #11 Valid: 79.4
             
#             [ our_models["DenseNet"], our_models["Inception-Fine"], our_models["InceptionResNet"]],                                     #12 Valid: 80.2
            
#             #4's
#             [ our_models["DenseNet"], our_models["Xception"], our_models["InceptionResNet"], our_models["Inception"]],                  #13 Valid: 82.2
             
#             [ our_models["DenseNet"], our_models["Xception"], our_models["InceptionResNet"], our_models["Inception-Fine"]],             #14 Valid: 82.1
             
#             [ our_models["DenseNet"], our_models["Xception-Fine"], our_models["InceptionResNet"], our_models["Inception"]],             #15 Valid: 81.3

#             [ our_models["DenseNet"], our_models["Xception-Fine"], our_models["InceptionResNet"], our_models["Inception-Fine"]],        #16 Valid: 81.5
            
#             #DenseNetFine 2's
#             [ our_models["DenseNet-Fine"], our_models["Xception"] ],                                                                    #17 Valid: 80.5

#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"] ],                                                               #18 Valid: 80.6

#             [ our_models["DenseNet-Fine"], our_models["InceptionResNet"] ],                                                             #19 Valid: 77.8

#             [ our_models["DenseNet-Fine"], our_models["Inception"] ],                                                                   #20 Valid: 78.2

#             [ our_models["DenseNet-Fine"], our_models["Inception-Fine"] ],                                                              #21 Valid: 78.7

#             #DenseNetFine 3's
#             [ our_models["DenseNet-Fine"], our_models["Xception"], our_models["InceptionResNet"]],                                      #22 Valid: 80.9

#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"], our_models["InceptionResNet"]],                                 #23 Valid: 80.4

#             [ our_models["DenseNet-Fine"], our_models["Xception"], our_models["Inception"]],                                            #24 Valid: 80.9
            
#             [ our_models["DenseNet-Fine"], our_models["Xception"], our_models["Inception-Fine"]],                                       #25 Valid: 81.3

#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"], our_models["Inception"]],                                       #26 Valid: 80.9
            
#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"], our_models["Inception-Fine"]],                                  #27 Valid: 81.4
            
#             [ our_models["DenseNet-Fine"], our_models["Inception"], our_models["InceptionResNet"]],                                     #28 Valid: 78.2
             
#             [ our_models["DenseNet-Fine"], our_models["Inception-Fine"], our_models["InceptionResNet"]],                                #29 Valid: 78.9

#             #DenseNetFine 4's
#             [ our_models["DenseNet-Fine"], our_models["Xception"], our_models["InceptionResNet"], our_models["Inception"]],             #30 Valid: 81.4
             
#             [ our_models["DenseNet-Fine"], our_models["Xception"], our_models["InceptionResNet"], our_models["Inception-Fine"]],        #31 Valid: 80.9
             
#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"], our_models["InceptionResNet"], our_models["Inception"]],        #32 Valid: 80.5

#             [ our_models["DenseNet-Fine"], our_models["Xception-Fine"], our_models["InceptionResNet"], our_models["Inception-Fine"]],   #33 Valid: 80.7

#         ]

# Valid_Acc = []

# for i in range(len(tryModels)):
#     ChoosenModel = i

#     ensemble_model = ensemble( tryModels[ChoosenModel], model_input)
#     ensemble_model.compile(loss='categorical_crossentropy',
#                 optimizer=optimizer,
#                 metrics=['accuracy'])

#     loss_val, acc_val = ensemble_model.evaluate(valid_X, valid_y, verbose=1)
#     print("Model #%d --> Validation: accuracy = %f  ;  loss_v = %f" % (ChoosenModel, acc_val, loss_val))
    
#     Valid_Acc.append(acc_val)

In [0]:
# for i in range(len(Valid_Acc)):
#     print("Model:" + str(i) + " --> " + str(Valid_Acc[i])[0:6])

Model:0 --> 0.8009
Model:1 --> 0.7990
Model:2 --> 0.8040
Model:3 --> 0.7929
Model:4 --> 0.7940
Model:5 --> 0.8050
Model:6 --> 0.8000
Model:7 --> 0.7990
Model:8 --> 0.8069
Model:9 --> 0.8000
Model:10 --> 0.7960
Model:11 --> 0.8029
Model:12 --> 0.8019
Model:13 --> 0.8090
Model:14 --> 0.8069
Model:15 --> 0.8040
Model:16 --> 0.8059
Model:17 --> 0.8050
Model:18 --> 0.7929
Model:19 --> 0.7900
Model:20 --> 0.7870
Model:21 --> 0.7900
Model:22 --> 0.8100
Model:23 --> 0.7990
Model:24 --> 0.8040
Model:25 --> 0.8040
Model:26 --> 0.8009
Model:27 --> 0.7960
Model:28 --> 0.7950
Model:29 --> 0.7929
Model:30 --> 0.8119
Model:31 --> 0.8069
Model:32 --> 0.8029
Model:33 --> 0.8050


# Real Test

This part is for kaggle Submissions.

In [0]:
testing_data = []

testing_data = pickle.load(open(test_database, "rb"))

print("Length: ", len(testing_data))

testing_data = np.array(testing_data).reshape(-1,IMG_SIZE,IMG_SIZE,3)

testing_data = testing_data / 255.0

Length:  5000


In [0]:
predictions = np.argmax(ensemble_model.predict(testing_data),-1)

In [0]:
results = []
for i in range(len(predictions)):
    results.append(predictions[i]+1)

In [0]:
print(results)

# Printing Data to CSV

In [0]:
from google.colab import files

fileName = "ENTER NAME HERE.csv"

def printResult(images,labels): 
    file = open(fileName,"w")

    if(len(images) != len(labels)):
        print("Error with the dimensions.")
        return False
    else:
        file.write("Id,Category\n")
        for i in range(len(images)):
            file.write( str(images[i]) + "," + str(labels[i]) + "\n")
        return True

csv_images = []
for i in range(10001,15001):
	csv_images.append("Image_" + str(i))

if( printResult(csv_images,results) ):
    files.download(filename= fileName)
else:
    print("Cannot download...")